# Steven Acevez 862397425

# Kenny Courser 861136315 

# Nayri Tagmazian 861260091

# Physics 243 Final Project Code

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from imblearn.over_sampling import SMOTE

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

import tensorflow as tf
from tensorflow import keras
from keras.utils import to_categorical
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.metrics import make_scorer, accuracy_score
from sklearn.model_selection import RandomizedSearchCV
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers import Adam, SGD, Adagrad, Adadelta, RMSprop
from tensorflow.keras.metrics import Accuracy, CategoricalAccuracy, SparseCategoricalAccuracy

import warnings
warnings.filterwarnings('ignore')

2023-08-24 00:14:42.082182: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
df = pd.read_csv('WineQT.csv')
df.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality,Id
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5,0
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5,1
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5,2
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6,3
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5,4


In [3]:
df = df.drop('Id', axis=1)

In [4]:
df.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5


In [5]:
df.shape

(1143, 12)

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1143 entries, 0 to 1142
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   fixed acidity         1143 non-null   float64
 1   volatile acidity      1143 non-null   float64
 2   citric acid           1143 non-null   float64
 3   residual sugar        1143 non-null   float64
 4   chlorides             1143 non-null   float64
 5   free sulfur dioxide   1143 non-null   float64
 6   total sulfur dioxide  1143 non-null   float64
 7   density               1143 non-null   float64
 8   pH                    1143 non-null   float64
 9   sulphates             1143 non-null   float64
 10  alcohol               1143 non-null   float64
 11  quality               1143 non-null   int64  
dtypes: float64(11), int64(1)
memory usage: 107.3 KB


In [7]:
df['quality'].value_counts()

5    483
6    462
7    143
4     33
8     16
3      6
Name: quality, dtype: int64

In [8]:
X = df.drop(columns=['quality'])
y = df['quality']

In [9]:
# Create a SMOTE instance
smote = SMOTE(random_state=42)

# Apply SMOTE to the training data
X_sm, y_sm = smote.fit_resample(X, y)

In [10]:
y_sm.shape

(2898,)

In [11]:
# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X_sm, y_sm, test_size=0.25, random_state=42)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [21]:
# Define the build function for the KerasClassifier
def build_classifier(layers, hidden_units, activation, optimizer, metrics):
    classifier = Sequential()
    classifier.add(Dense(units=hidden_units, activation=activation, input_dim=X_train.shape[1]))
    for _ in range(layers - 1):
        classifier.add(Dense(units=hidden_units, activation=activation))
    classifier.add(Dense(units=len(y.unique()), activation='softmax'))
    classifier.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=metrics)
    return classifier

# Create a KerasClassifier with build_classifier function
classifier = KerasClassifier(build_fn=build_classifier)

# Define hyperparameters grid for GridSearchCV
param_grid = {
    'layers': [1, 2, 3],
    'hidden_units': [16, 32, 64, 128, 256],
    'activation': ['relu', 'sigmoid', 'tanh', 'linear'],
    'batch_size': [16, 32, 64, 128, 256],
    'optimizer': ['adam', 'sgd', 'adagrad', 'adadelta', 'rmsprop'],
    'metrics' : [Accuracy(), CategoricalAccuracy(), SparseCategoricalAccuracy()]
}

# Perform grid search
grid_search = RandomizedSearchCV(estimator=classifier, param_distributions=param_grid, scoring='accuracy', cv=3)
grid_search = grid_search.fit(X_train_scaled, y_train)

# Get the best model and its accuracy
best_model = grid_search.best_estimator_
best_accuracy = grid_search.best_score_

print("Best Model Hyperparameters:")
print(best_model.get_params())
print("Best Accuracy:", best_accuracy)

136/136 [==============================] - 1s 2ms/step - loss: 1.0271 - sparse_categorical_accuracy: 0.5780
Best Model Hyperparameters:
{'optimizer': 'rmsprop', 'metrics': <keras.metrics.metrics.SparseCategoricalAccuracy object at 0x7fed094322d0>, 'layers': 3, 'hidden_units': 256, 'batch_size': 16, 'activation': 'relu', 'build_fn': <function build_classifier at 0x7fed093c89e0>}
Best Accuracy: 0.6309201752714803


In [22]:
# Evaluate the best model on the test set
y_pred = best_model.predict(X_test_scaled)
test_accuracy = accuracy_score(y_test, y_pred)
print("Test Accuracy:", test_accuracy)

# Feature Importance Analysis:
# For simplicity, feature weights are calculated from the final layer of the neural network.
feature_weights = best_model.model.layers[-1].get_weights()[0]
feature_importances = np.sum(np.abs(feature_weights), axis=1)
sorted_features = np.argsort(feature_importances)[::-1]

print("Feature Importance Ranking:")
for idx, feature_name in enumerate(X.columns):
    print(f"{feature_name}: {feature_importances[idx]}")

23/23 [==============================] - 0s 1ms/step
Test Accuracy: 0.6537931034482759
Feature Importance Ranking:
fixed acidity: 0.2636304795742035
volatile acidity: 0.4196866750717163
citric acid: 0.44442009925842285
residual sugar: 0.46308547258377075
chlorides: 0.7414376139640808
free sulfur dioxide: 0.631073534488678
total sulfur dioxide: 0.4168376624584198
density: 0.4321860671043396
pH: 0.4619922637939453
sulphates: 0.3559524714946747
alcohol: 0.4613977074623108
